# Steps Analysis (02/17/17 - 10/24/18)

In [231]:
from main_clean_functions import main_clean_function
from main_clean_functions import create_combined_daily_df
from common_cleaning_functions import print_remaining
from common_cleaning_functions import print_all_info
from stats_functions import get_stats_day_week_month
from stats_functions import rolling_day_df

In [232]:
steps_df, dist_df, floors_df = main_clean_function('../data/export.xml')

In [238]:
combined_df = create_combined_daily_df(steps_df, dist_df, floors_df)

In [244]:
get_stats_day_week_month(combined_df, 'Day of Month', 'floors')

,date_year,std_steps,std_dist,std_flrs,std_fps,mean_steps,mean_dist,mean_flrs,mean_fps,steps_score,dist_score,floors_score
0,06-2018,3150.389516,1.508999,12.545030,0.134483,9328.900000,3.934987,15.600000,2.207649,27624.639677,10.261191,19.398917
1,12-2018,2836.667011,1.263161,12.764534,0.177177,9620.800000,4.014932,14.400000,2.199671,32629.770174,12.761379,16.245011
2,13-2018,2265.118981,0.996697,10.263744,0.167894,8269.000000,3.399838,12.300000,2.171076,30186.653134,11.597201,14.740235
3,28-2018,2438.330941,1.046542,11.532563,0.136411,10783.444444,4.506958,12.666667,2.207367,47689.455164,19.409320,13.912298
4,18-2018,3687.358779,1.527527,10.168033,0.157459,10239.900000,4.301952,11.500000,2.220768,28436.492972,12.115528,13.006449
5,14-2018,2704.139150,1.067185,6.995236,0.139688,8627.100000,3.588143,9.400000,2.221751,27523.307889,12.064238,12.631453
6,02-2018,2569.703390,1.190539,12.203825,0.132136,9109.200000,3.783898,12.400000,2.179651,32290.701321,12.026391,12.599329
7,30-2018,4461.711025,2.093206,12.950455,0.209211,7284.125000,3.033878,12.500000,2.126708,11891.957306,4.397283,12.065213
8,25-2018,1797.198870,0.884646,10.970921,0.118577,9065.444444,3.862259,11.111111,2.240310,45727.985007,16.862155,11.253092
9,21-2018,2392.211520,0.843708,13.273198,0.124093,10107.200000,4.237922,12.200000,2.230699,42703.369244,21.286975,11.213575
